In [1]:
import time
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from holidays.countries import US
import numpy as np

### DEFINE

In [2]:
# 讀取檔案位置

# 初始年月份
set_data_year = 2021 #start_year
set_data_month = 1 #start_month
end_year = 2021
end_month = 12

# 檔案位置
Site = './data/'

# 檔案輸出位置
custom_output_path_dfTime = './dfTime/test'
custom_output_path_dfTLC = './dfTLC/test'

# 年月
start_year = set_data_year
start_month = set_data_month

datasite_fhvhv = []
datasite_yellow = []
for year in range(start_year, end_year+1):  # 修改为你的年份范围
    if year == start_year:
        if year < end_year :
            for month in range(start_month, 13):  # 生成1到12的月份
                filename = f'{Site}fhvhv_tripdata_{year}-{month:02}.parquet'
                datasite_fhvhv.append(filename)
                filename = f'{Site}yellow_tripdata_{year}-{month:02}.parquet'
                datasite_yellow.append(filename)                
        elif year == end_year:
            for month in range(start_month, end_month+1):  # 生成1到12的月份
                filename = f'{Site}fhvhv_tripdata_{year}-{month:02}.parquet'
                datasite_fhvhv.append(filename)    
                filename = f'{Site}yellow_tripdata_{year}-{month:02}.parquet'
                datasite_yellow.append(filename)   
    elif year > start_year:
        if year < end_year :
            for month in range(1, 13):  # 生成1到12的月份
                filename = f'{Site}fhvhv_tripdata_{year}-{month:02}.parquet'
                datasite_fhvhv.append(filename)
                filename = f'{Site}yellow_tripdata_{year}-{month:02}.parquet'
                datasite_yellow.append(filename)   
        elif year == end_year:
            for month in range(1, end_month+1):  # 生成1到12的月份
                filename = f'{Site}fhvhv_tripdata_{year}-{month:02}.parquet'
                datasite_fhvhv.append(filename)         
                filename = f'{Site}yellow_tripdata_{year}-{month:02}.parquet'
                datasite_yellow.append(filename)   



print("done時間:", datetime.datetime.now())

done時間: 2023-08-16 15:58:43.511190


### 留下需要特徵 & 移除不在當月資料
    HVFHV: 'hvfhs_license_num','pickup_datetime','dropoff_datetime','PULocationID','DOLocationID','trip_miles','trip_time','base_passenger_fare'
    
    Yellow: 'tpep_pickup_datetime','tpep_dropoff_datetime','trip_distance','PULocationID','DOLocationID','fare_amount'

In [3]:
# 記錄開始時間
start_time = time.time()


# 建立一個空的DataFrame來存放合併後的數據
df_fhvhv = pd.DataFrame()
df_yellow = pd.DataFrame()

data_year = set_data_year
data_month = set_data_month
temp0_total_fhvfv = 0
temp1_total_fhvfv = 0
temp0_total_yellow = 0
temp1_total_yellow = 0
# 逐一讀取並合併所有Parquet檔案
print('fhvfv==初始筆數=剩餘筆數=移除佔比')
for file in datasite_fhvhv:
    # 使用pandas的read_parquet函數讀取Parquet檔案並將其轉換為DataFrame
    df = pd.read_parquet(file)
    temp_0 = df.shape[0]
    temp0_total_fhvfv += temp_0
    # 留下需要的欄位
    df =  df[['hvfhs_license_num','pickup_datetime','dropoff_datetime','PULocationID','DOLocationID','trip_miles','trip_time','shared_match_flag']]
    # 移除非當月的data
    df =  df[(df['pickup_datetime'].dt.year == data_year) & (df['pickup_datetime'].dt.month == data_month)]
    temp_1 = df.shape[0]
    temp1_total_fhvfv += temp_1
    # 查看剩餘資料量
    if temp_0 > temp_1 :
        print(data_year,'=',data_month,'=',temp_0,'=',temp_1,round(100-(temp_1/temp_0*100),5),"%")
    else:
        print(data_year,'=',data_month,'=',temp_0,'=',temp_1,'=0%')
    # 年月處理
    if data_month == 12:
        data_year += 1
        data_month = 1
    else :
        data_month += 1
    # 將數據合併到已有的DataFrame中
    df_fhvhv = pd.concat([df_fhvhv, df], ignore_index=True)
    
    

data_year = set_data_year
data_month = set_data_month
print('yellow==初始筆數=剩餘筆數=移除佔比')
for file in datasite_yellow:
    # 使用pandas的read_parquet函數讀取Parquet檔案並將其轉換為DataFrame
    df = pd.read_parquet(file)
    temp_0 = df.shape[0]
    temp0_total_yellow += temp_0
    # 留下需要的欄位
    df =  df[['tpep_pickup_datetime','tpep_dropoff_datetime','trip_distance','PULocationID','DOLocationID']]
    # 移除非當月的data
    df =  df[(df['tpep_pickup_datetime'].dt.year == data_year) & (df['tpep_pickup_datetime'].dt.month == data_month)]
    temp_1 = df.shape[0]
    temp1_total_yellow += temp_1
    # 查看剩餘資料量
    if temp_0 > temp_1 :
        print(data_year,'=',data_month,'=',temp_0,'=',temp_1,'=',round(100-(temp_1/temp_0*100),5),"%")
    else:
        print(data_year,'=',data_month,'=',temp_0,'=',temp_1,'=0%')        
    # 年月處理
    if data_month == 12:
        data_year += 1
        data_month = 1
    else :
        data_month += 1
    # 將數據合併到已有的DataFrame中
    df_yellow = pd.concat([df_yellow, df], ignore_index=True)
    
    
print(f'fhvfv={start_year}=初始總筆數=剩餘總筆數=移除佔比')
if temp0_total_fhvfv > temp1_total_fhvfv :
    print('\==',temp0_total_fhvfv,'=',temp1_total_fhvfv,'=',round(100-(temp1_total_fhvfv/temp0_total_fhvfv*100),5),"%")
else:
    print('\==',temp0_total_fhvfv,'=',temp1_total_fhvfv,'=0%')

print(f'yellow={start_year}=初始總筆數=剩餘總筆數=移除佔比')
if temp0_total_yellow > temp1_total_yellow :
    print('\==',temp0_total_yellow,'=',temp1_total_yellow,'=',round(100-(temp1_total_yellow/temp0_total_yellow*100),5),"%")
else:
    print('\==',temp0_total_yellow,'=',temp1_total_yellow,'=0%')

# 記錄結束時間
end_time = time.time()
# 計算程式執行花費的時間
execution_time = end_time - start_time
print("\n程式執行花費的時間：", round(execution_time,2), "秒")


#清理暫存
del data_month,data_year,df,temp_0,temp_1,temp0_total_yellow,temp1_total_yellow
print("done時間:", datetime.datetime.now())

fhvfv==初始筆數=剩餘筆數=移除佔比
2021 = 1 = 11908468 = 11908468 =0%
2021 = 2 = 11613942 = 11613942 =0%
2021 = 3 = 14227393 = 14227393 =0%
2021 = 4 = 14111371 = 14111371 =0%
2021 = 5 = 14719171 = 14719171 =0%
2021 = 6 = 14961892 = 14961892 =0%
2021 = 7 = 15027174 = 15027174 =0%
2021 = 8 = 14499696 = 14499696 =0%
2021 = 9 = 14886055 = 14886055 =0%
2021 = 10 = 16545356 = 16545356 =0%
2021 = 11 = 16041639 = 16041639 =0%
2021 = 12 = 16054495 = 16054495 =0%
yellow==初始筆數=剩餘筆數=移除佔比
2021 = 1 = 1369769 = 1369745 = 0.00175 %
2021 = 2 = 1371709 = 1371678 = 0.00226 %
2021 = 3 = 1925152 = 1925119 = 0.00171 %
2021 = 4 = 2171187 = 2171145 = 0.00193 %
2021 = 5 = 2507109 = 2507015 = 0.00375 %
2021 = 6 = 2834264 = 2834141 = 0.00434 %
2021 = 7 = 2821746 = 2821616 = 0.00461 %
2021 = 8 = 2788757 = 2788231 = 0.01886 %
2021 = 9 = 2963793 = 2963688 = 0.00354 %
2021 = 10 = 3463504 = 3463393 = 0.0032 %
2021 = 11 = 3472949 = 3472822 = 0.00366 %
2021 = 12 = 3214369 = 3214261 = 0.00336 %
fhvfv=2021=初始總筆數=剩餘總筆數=移除佔比
\== 174596

In [4]:
df_fhvhv

,hvfhs_license_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,shared_match_flag
0,HV0003,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,5.260,923,N
1,HV0003,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,3.650,1382,N
2,HV0003,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,3.510,849,N
3,HV0003,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,0.740,179,N
4,HV0003,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,9.200,1228,N
...,...,...,...,...,...,...,...,...
174596647,HV0005,2021-12-31 23:23:37,2021-12-31 23:29:50,180,197,1.516,373,N
174596648,HV0005,2021-12-31 23:33:44,2021-12-31 23:37:34,216,216,0.935,230,N
174596649,HV0005,2021-12-31 23:59:46,2022-01-01 00:11:21,132,134,6.358,745,N
174596650,HV0003,2021-12-31 23:27:13,2021-12-31 23:41:28,235,69,2.410,855,N


In [5]:
df_yellow

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID
0,2021-01-01 00:30:10,2021-01-01 00:36:12,2.10,142,43
1,2021-01-01 00:51:20,2021-01-01 00:52:19,0.20,238,151
2,2021-01-01 00:43:30,2021-01-01 01:11:06,14.70,132,165
3,2021-01-01 00:15:48,2021-01-01 00:31:01,10.60,138,132
4,2021-01-01 00:31:49,2021-01-01 00:48:21,4.94,68,33
...,...,...,...,...,...
30902849,2021-12-31 23:18:02,2021-12-31 23:32:19,3.99,145,107
30902850,2021-12-31 23:46:00,2022-01-01 00:07:00,5.96,236,158
30902851,2021-12-31 23:46:00,2022-01-01 00:04:00,4.99,48,13
30902852,2021-12-31 23:53:00,2022-01-01 00:02:00,1.36,75,75


### 特徵整理,統一命名
HVFHV:'hvfhs_license_num','pickup_datetime','dropoff_datetime','PULocationID','DOLocationID','trip_miles','trip_time'
Yellow:'tpep_pickup_datetime','tpep_dropoff_datetime','trip_distance','PULocationID','DOLocationID'


    newname (HVFHV/Yellow/Green)
    1.TLC 許可證號 : Name(hvfhs_license_num/--/--)
        *改為名稱標記 HV0003-uber/HV0005-lyft/yellow
    2.上車時間 : BoardingTime (pickup_datetime/tpep_pickup_datetime/lpep_pickup_datetime)
    3.下車時間 : DisembarkingTime (dropoff_datetime/tpep_dropoff_datetime/lpep_dropoff_datetime)
    4.上車地點ID : PULocationID (PULocationID/PULocationID/PULocationID)
    5.下車地點ID : DOLocationID (DOLocationID/DOLocationID/DOLocationID)
    6.行駛里程 : TripMiles (trip_miles/trip_distance/trip_distance)
    7.行駛時間(秒) : TripTime (trip_time/--/--)
        *驗證是否正確?
        *下車時間-上車時間
    8.共乘 : shared_match_flag (shared_match_flag/--/--)
    9.節日 : is_holiday(--/--/--)
    10.星期 : weekday(--/--/--)

In [6]:
# 記錄開始時間
start_time = time.time()

df_TLC = pd.DataFrame()
#名稱統一
rename_dict_fhvhv = {'hvfhs_license_num':'Name','pickup_datetime': 'BoardingTime', 'dropoff_datetime': 'DisembarkingTime', 'PULocationID': 'PULocationID','DOLocationID':'DOLocationID', 'trip_miles': 'TripMiles', 'trip_time': 'TripTime', 'base_passenger_fare': 'BaseFare','shared_match_flag':'shared_match_flag'}
df_fhvhv.rename(columns=rename_dict_fhvhv, inplace=True)
rename_dict_yellow = {'tpep_pickup_datetime': 'BoardingTime', 'tpep_dropoff_datetime': 'DisembarkingTime', 'PULocationID': 'PULocationID','DOLocationID':'DOLocationID', 'trip_distance': 'TripMiles', 'fare_amount': 'BaseFare'}
df_yellow.rename(columns=rename_dict_yellow, inplace=True)

#fhvhv_Name
df_fhvhv.loc[df_fhvhv['Name'] == 'HV0003', 'Name'] = 'uber'
df_fhvhv.loc[df_fhvhv['Name'] == 'HV0005', 'Name'] = 'lyft'

#yellow 新增特徵 TripTime,Name
df_yellow['TripTime'] = (df_yellow['DisembarkingTime'] - df_yellow['BoardingTime']).dt.total_seconds()
df_yellow['Name'] = 'yellow'
df_yellow['shared_match_flag'] = 'N'
#yellow合併到fhvhv
df_yellow = df_yellow.reindex(columns = df_fhvhv.columns)
df_TLC = pd.concat([df_fhvhv, df_yellow], ignore_index=True)


# 記錄結束時間
end_time = time.time()
# 計算程式執行花費的時間
execution_time = end_time - start_time
print("程式執行花費的時間：", round(execution_time,2), "秒")

#清理暫存
del df_yellow ,df_fhvhv
print("done時間:", datetime.datetime.now())

程式執行花費的時間： 30.76 秒
done時間: 2023-08-16 16:13:27.596786


### 紀錄初始資料筆數

In [7]:
#紀錄初始資料筆數
init_df_TLC_shape = df_TLC.shape
init_df_uber_shape = df_TLC[df_TLC['Name']=='uber'].shape
init_df_lyft_shape = df_TLC[df_TLC['Name']=='lyft'].shape
init_df_yellow_shape = df_TLC[df_TLC['Name']=='yellow'].shape
print('Name=init_data_row=init_data_column')
print(f'TLC={init_df_TLC_shape[0]}={init_df_TLC_shape[1]}')
print(f'uber={init_df_uber_shape[0]}={init_df_uber_shape[1]}')
print(f'lyft={init_df_lyft_shape[0]}={init_df_lyft_shape[1]}')
print(f'yellow={init_df_yellow_shape[0]}={init_df_yellow_shape[1]}')

print("\ndone時間:", datetime.datetime.now())

Name=init_data_row=init_data_column
TLC=205499506=8
uber=126129064=8
lyft=47575769=8
yellow=30902854=8

done時間: 2023-08-16 16:14:33.154550


### 缺失值確認

In [8]:
print(df_TLC.isnull().sum())

Name                 0
BoardingTime         0
DisembarkingTime     0
PULocationID         0
DOLocationID         0
TripMiles            0
TripTime             0
shared_match_flag    0
dtype: int64


### 移除非目標車商

In [9]:
#查看目前有的車商
print(df_TLC['Name'].unique())

print("\ndone時間:", datetime.datetime.now())

['uber' 'lyft' 'HV0004' 'yellow']

done時間: 2023-08-16 16:15:40.787954


In [10]:
#筆數計算-2
print('HV0004=筆數=占比')
print('\=', df_TLC[(df_TLC['Name'] == 'HV0004')].shape[0],'=',round(df_TLC[(df_TLC['Name'] == 'HV0004')].shape[0] / init_df_TLC_shape[0]*100,4),'%')

print("\ndone時間:", datetime.datetime.now())

HV0004=筆數=占比
\= 891819 = 0.434 %

done時間: 2023-08-16 16:16:07.187213


In [11]:
#移除
df_TLC = df_TLC.drop(df_TLC[df_TLC['Name'] == 'HV0002'].index)
df_TLC = df_TLC.drop(df_TLC[df_TLC['Name'] == 'HV0004'].index)
print(df_TLC['Name'].unique())

print("\ndone時間:",datetime.datetime.now())

['uber' 'lyft' 'yellow']

done時間: 2023-08-16 16:17:24.763745


### 移除上車地點 PULocationID 未知 (264,265)

In [12]:
#筆數計算
print('264筆數=265筆數=總占比') 
print(df_TLC[(df_TLC['PULocationID'] == 264)].shape[0],'=', df_TLC[(df_TLC['PULocationID'] == 265)].shape[0],'=', 
       round(df_TLC[(df_TLC['PULocationID'] == 264) | (df_TLC['PULocationID'] == 265)].shape[0] / init_df_TLC_shape[0]*100,4),'%'  )

print("\ndone時間:", datetime.datetime.now())

264筆數=265筆數=總占比
230574 = 150795 = 0.1856 %

done時間: 2023-08-16 16:18:06.907299


In [13]:
df_TLC = df_TLC[~df_TLC['PULocationID'].isin([264, 265])]

#check-必須為0
print('上車未知剩餘筆數 = ',df_TLC[(df_TLC['PULocationID'] == 264) | (df_TLC['PULocationID'] == 265)].shape[0])
print("\ndone時間:", datetime.datetime.now())

上車未知剩餘筆數 =  0

done時間: 2023-08-16 16:18:31.948141


### 移除不符合的共乘資料
    UBER 2020/3/17-2022/6/20
    LYFT 2020/3/17-2021/8/18
         2023/5/12-

In [14]:
#uber,lyft
# 定義起始日期和結束日期
uber_start_date = pd.to_datetime('2020-03-17')
uber_end_date = pd.to_datetime('2022-06-20')
lyft_start_date = pd.to_datetime('2020-03-17')
lyft_end_date = pd.to_datetime('2021-08-18')
lyft_start_date1 = pd.to_datetime('2023-05-12')

#筆數計算
temp_uber = ((df_TLC['Name'] == 'uber') & (df_TLC['shared_match_flag'] == 'Y') & (df_TLC['BoardingTime'].between(uber_start_date, uber_end_date))).sum()

temp_lyft = ((df_TLC['Name'] == 'lyft') & (df_TLC['shared_match_flag'] == 'Y') & ((df_TLC['BoardingTime'].between(lyft_start_date, lyft_end_date))|(df_TLC['BoardingTime'] >= lyft_start_date1 ))).sum()
print('/=uber=lyft')
print('data=',temp_uber,'=',temp_lyft)
print('占比=',round(temp_uber / init_df_uber_shape[0]*100,4),'%','=',round(temp_lyft / init_df_lyft_shape[0]*100,4),'%' )

print("\ndone時間:", datetime.datetime.now())

#清理暫存
del temp_uber ,temp_lyft

/=uber=lyft
data= 0 = 52
占比= 0.0 % = 0.0001 %

done時間: 2023-08-16 16:19:13.403314


In [16]:
# 過濾條件：shared_match_flag為Y且BoardingTime介於指定日期之間
df_TLC = df_TLC[~ ((df_TLC['Name'] == 'uber') & (df_TLC['shared_match_flag'] == 'Y') & (df_TLC['BoardingTime'].between(uber_start_date, uber_end_date)))]
df_TLC = df_TLC[~ ((df_TLC['Name'] == 'lyft') & (df_TLC['shared_match_flag'] == 'Y') & ((df_TLC['BoardingTime'].between(lyft_start_date, lyft_end_date))|(df_TLC['BoardingTime'] >= lyft_start_date1 )))]


#check-必須為0
print('uber不符合的共乘資料剩餘筆數 = ',df_TLC[(df_TLC['Name'] == 'uber') & (df_TLC['shared_match_flag'] == 'Y') & (df_TLC['BoardingTime'].between(uber_start_date, uber_end_date))].shape[0]  )
print('lyft不符合的共乘資料剩餘筆數 = ',df_TLC[(df_TLC['Name'] == 'lyft') & (df_TLC['shared_match_flag'] == 'Y') & ((df_TLC['BoardingTime'].between(lyft_start_date, lyft_end_date))|(df_TLC['BoardingTime'] >= lyft_start_date1 ))].shape[0]  )
print("\ndone時間:", datetime.datetime.now())

#清理暫存
del uber_start_date ,uber_end_date,lyft_start_date,lyft_end_date,lyft_start_date1

uber不符合的共乘資料剩餘筆數 =  0
lyft不符合的共乘資料剩餘筆數 =  0

done時間: 2023-08-16 14:29:59.209343


###　不處理  trip_time != DisembarkingTime - BoardingTime
### 移除 TripTime <= 0

In [17]:
#記錄開始時間
start_time = time.time()

#印出資訊
temp_uber = ((df_TLC['Name'] == 'uber') & (df_TLC['TripTime'] <= 0)).sum()
temp_lyft = ((df_TLC['Name'] == 'lyft') & (df_TLC['TripTime'] <= 0)).sum()
temp_yellow = ((df_TLC['Name'] == 'yellow') & (df_TLC['TripTime'] <= 0)).sum()

print('\=uber=lyft=yellow')
print('data=',temp_uber,'=',temp_lyft,'=',temp_yellow)
print('占比=',round(temp_uber / init_df_uber_shape[0]*100,5),'%='
              ,round(temp_lyft / init_df_lyft_shape[0]*100,5),'%='
              ,round(temp_yellow / init_df_yellow_shape[0]*100,5),'%' )

#記錄結束時間
end_time = time.time()
#計算程式執行花費的時間
execution_time = end_time - start_time
print("\n程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())

\=uber=lyft=yellow
data= 3 = 83 = 19536
占比= 0.0 %= 0.00017 %= 0.06322 %

程式執行花費的時間： 23.97 秒
done時間: 2023-08-16 14:30:33.895995


In [18]:
#移除 (df_TLC['TripTime'] <= 0)
df_TLC = df_TLC.drop(df_TLC[df_TLC['TripTime'] <= 0].index)
#check-必須為0
print('TripTime<=0剩餘筆數:',(df_TLC['TripTime'] <= 0).sum())
print("\ndone時間:", datetime.datetime.now())

TripTime<=0剩餘筆數: 0

done時間: 2023-08-16 14:32:03.266854


### 移除TripMiles <= 0

In [19]:
#記錄開始時間
start_time = time.time()

#印出資訊
temp_uber = ((df_TLC['Name'] == 'uber') & (df_TLC['TripMiles'] <= 0)).sum()
temp_lyft = ((df_TLC['Name'] == 'lyft') & (df_TLC['TripMiles'] <= 0)).sum()
temp_yellow = ((df_TLC['Name'] == 'yellow') & (df_TLC['TripMiles'] <= 0)).sum()

print('\=uber=lyft=yellow')
print('data=',temp_uber,'=',temp_lyft,'=',temp_yellow)
print('占比=',round(temp_uber / init_df_uber_shape[0]*100,5),'%='
              ,round(temp_lyft / init_df_lyft_shape[0]*100,5),'%='
              ,round(temp_yellow / init_df_yellow_shape[0]*100,5),'%' )


#記錄結束時間
end_time = time.time()
#計算程式執行花費的時間
execution_time = end_time - start_time
print("\n程式執行花費的時間：", round(execution_time,2), "秒")

del temp_uber,temp_lyft,temp_yellow
print("done時間:",datetime.datetime.now())

\=uber=lyft=yellow
data= 27286 = 5318 = 364484
占比= 0.02163 %= 0.01118 %= 1.17945 %

程式執行花費的時間： 24.02 秒
done時間: 2023-08-16 14:33:04.174589


In [20]:
#移除 (df_TLC['TripMiles'] <= 0)
df_TLC = df_TLC.drop(df_TLC[df_TLC['TripMiles'] <= 0].index)
#check-必須為0
print('TripMiles<=0剩餘筆數:',(df_TLC['TripMiles'] <= 0).sum())
print("\ndone時間:", datetime.datetime.now())

TripMiles<=0剩餘筆數: 0

done時間: 2023-08-16 14:34:09.189626


### 新增節日相關資訊

In [21]:
# from holidays.calendars import gregorian
class AllHolidays(US):
    def _populate(self, year):
        super()._populate(year)
        self.pop_named("Lincoln's Birthday")
        self.pop_named("Susan B. Anthony Day")
        
        self._add_holiday("Valentine's Day", 2, 14)
        self._add_holiday("St.Patrick's Day", 3, 17)
        self._add_holiday("Halloween", 10, 31)
        
#        self._add_holiday("Thanksgiving weekend", 
#                          self._get_nth_weekday_of_month(month=11, n=4, weekday=4))
#        self._add_holiday("Thanksgiving weekend", 
#                          self._get_nth_weekday_of_month(month=11, n=4, weekday=5))
#        self._add_holiday("Thanksgiving weekend", 
#                          self._get_nth_weekday_of_month(month=11, n=4, weekday=6))
        
        self._add_holiday("Christmas Eve", 12, 24)
        self._add_holiday("Christmas season", 12, 26)
        self._add_holiday("Christmas season", 12, 27)
        self._add_holiday("Christmas season", 12, 28)
        self._add_holiday("Christmas season", 12, 29)
        self._add_holiday("Christmas season", 12, 30)
        
        self._add_holiday("New Year's Eve", 12, 31)

# 實例化AllHolidays
ny_holidays_all = AllHolidays(subdiv='NY', years=[2019, 2020, 2021, 2022, 2023])

# 新增2019~2023年的復活節、復活節星期一
ny_holidays_all[datetime.date(2019, 4, 21)] = "Easter"
ny_holidays_all[datetime.date(2019, 4, 22)] = "Easter Monday"
ny_holidays_all[datetime.date(2020, 4, 12)] = "Easter"
ny_holidays_all[datetime.date(2020, 4, 13)] = "Easter Monday"
ny_holidays_all[datetime.date(2021, 4, 4)] = "Easter"
ny_holidays_all[datetime.date(2021, 4, 5)] = "Easter Monday"
ny_holidays_all[datetime.date(2022, 4, 17)] = "Easter"
ny_holidays_all[datetime.date(2022, 4, 18)] = "Easter Monday"
ny_holidays_all[datetime.date(2023, 4, 9)] = "Easter"
ny_holidays_all[datetime.date(2023, 4, 10)] = "Easter Monday"

# 新增2019~2023感恩節
ny_holidays_all[datetime.date(2019,11, 28)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2019,11, 29)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2019,11, 30)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2020,11, 26)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2020,11, 27)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2020,11, 28)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2021,11, 25)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2021,11, 26)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2021,11, 27)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2022,11, 24)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2022,11, 25)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2022,11, 26)] =  "Thanksgiving weekend"
ny_holidays_all[datetime.date(2023,11, 23)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2023,11, 24)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2023,11, 25)] = "Thanksgiving weekend"

# 查看
for i in sorted(ny_holidays_all.items()):
    print(i)

del i

(datetime.date(2019, 1, 1), "New Year's Day")
(datetime.date(2019, 1, 21), 'Martin Luther King Jr. Day')
(datetime.date(2019, 2, 14), "Valentine's Day")
(datetime.date(2019, 2, 18), "Washington's Birthday")
(datetime.date(2019, 3, 17), "St.Patrick's Day")
(datetime.date(2019, 4, 21), 'Easter')
(datetime.date(2019, 4, 22), 'Easter Monday')
(datetime.date(2019, 5, 27), 'Memorial Day')
(datetime.date(2019, 7, 4), 'Independence Day')
(datetime.date(2019, 9, 2), 'Labor Day')
(datetime.date(2019, 10, 14), 'Columbus Day')
(datetime.date(2019, 10, 31), 'Halloween')
(datetime.date(2019, 11, 5), 'Election Day')
(datetime.date(2019, 11, 11), 'Veterans Day')
(datetime.date(2019, 11, 28), 'Thanksgiving; Thanksgiving weekend')
(datetime.date(2019, 11, 29), 'Thanksgiving weekend')
(datetime.date(2019, 11, 30), 'Thanksgiving weekend')
(datetime.date(2019, 12, 24), 'Christmas Eve')
(datetime.date(2019, 12, 25), 'Christmas Day')
(datetime.date(2019, 12, 26), 'Christmas season')
(datetime.date(2019, 12, 

In [22]:
# 增加所有節日標籤
# all_data為Uber, Lyft, Yellow合併後的DataFrame，再請自行修改名稱
df_TLC['is_holiday'] = np.where(df_TLC['BoardingTime'].dt.date.isin(ny_holidays_all), True, False)
print(df_TLC['is_holiday'].value_counts())

print('-------------')
print('Name=True=False')
print('uber=',((df_TLC['Name'] == 'uber') & (df_TLC['is_holiday'] == True)).sum(),'=',((df_TLC['Name'] == 'uber') & (df_TLC['is_holiday'] == False)).sum())
print('lyft=',((df_TLC['Name'] == 'lyft') & (df_TLC['is_holiday'] == True)).sum(),'=',((df_TLC['Name'] == 'lyft') & (df_TLC['is_holiday'] == False)).sum())
print('yellow=',((df_TLC['Name'] == 'yellow') & (df_TLC['is_holiday'] == True)).sum(),'=',((df_TLC['Name'] == 'yellow') & (df_TLC['is_holiday'] == False)).sum())
print('Total=',(df_TLC['is_holiday'] == True).sum(),'=',(df_TLC['is_holiday'] == False).sum())

print("\ndone時間:", datetime.datetime.now())



is_holiday
False    189393488
True      14416068
Name: count, dtype: int64
-------------
Name=True=False
uber= 9015033 = 117080680
lyft= 3450289 = 44115042
yellow= 1950746 = 28197766
Total= 14416068 = 189393488

done時間: 2023-08-16 14:35:50.733519


### 新增星期特徵
    #星期幾 (用上車時間 BoardingTime)
    #(一,二,三,四,五,六,日) = (0,1,2,3,4,5,6)

In [23]:
#記錄開始時間
start_time = time.time()

df_TLC['weekday'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.weekday
df_TLC['date'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.date
df_TLC['year'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.year
df_TLC['month'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.month
df_TLC['day'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.day
df_TLC['hour'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.hour

#記錄結束時間
end_time = time.time()
#計算程式執行花費的時間
execution_time = end_time - start_time
print("程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())

程式執行花費的時間： 91.47 秒
done時間: 2023-08-16 14:37:46.790589


### 時間序列
    'Name','date','year','month','day','hour','weekday','PULocationID','DOLocationID',
    'TripMiles','TripTime','shared_match_flag'

In [25]:
#記錄開始時間
start_time = time.time()

dfTime_TLC = df_TLC.groupby(['Name','year','month','day',
                     'hour','weekday','PULocationID','is_holiday'
                  ]).size().reset_index(name="次數")

#記錄結束時間
end_time = time.time()
#計算程式執行花費的時間
execution_time = end_time - start_time
print("程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())

程式執行花費的時間： 52.74 秒
done時間: 2023-08-16 14:39:56.279741


In [26]:
print('row=column')
print(dfTime_TLC.shape[0],'=',dfTime_TLC.shape[1])
dfTime_TLC

row=column
5193750 = 9


,Name,year,month,day,hour,weekday,PULocationID,is_holiday,次數
0,lyft,2021,1,1,0,4,3,True,15
1,lyft,2021,1,1,0,4,4,True,41
2,lyft,2021,1,1,0,4,6,True,4
3,lyft,2021,1,1,0,4,7,True,64
4,lyft,2021,1,1,0,4,9,True,2
...,...,...,...,...,...,...,...,...,...
5193745,yellow,2021,12,31,23,4,255,True,4
5193746,yellow,2021,12,31,23,4,256,True,2
5193747,yellow,2021,12,31,23,4,261,True,10
5193748,yellow,2021,12,31,23,4,262,True,30


In [27]:
print('row=column')
print(df_TLC.shape[0],'=',df_TLC.shape[1])
df_TLC

row=column
203809556 = 15


,Name,BoardingTime,DisembarkingTime,PULocationID,DOLocationID,TripMiles,TripTime,shared_match_flag,is_holiday,weekday,date,year,month,day,hour
0,uber,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,5.26,923.0,N,True,4,2021-01-01,2021,1,1,0
1,uber,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,3.65,1382.0,N,True,4,2021-01-01,2021,1,1,0
2,uber,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,3.51,849.0,N,True,4,2021-01-01,2021,1,1,0
3,uber,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,0.74,179.0,N,True,4,2021-01-01,2021,1,1,0
4,uber,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,9.20,1228.0,N,True,4,2021-01-01,2021,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205499501,yellow,2021-12-31 23:18:02,2021-12-31 23:32:19,145,107,3.99,857.0,N,True,4,2021-12-31,2021,12,31,23
205499502,yellow,2021-12-31 23:46:00,2022-01-01 00:07:00,236,158,5.96,1260.0,N,True,4,2021-12-31,2021,12,31,23
205499503,yellow,2021-12-31 23:46:00,2022-01-01 00:04:00,48,13,4.99,1080.0,N,True,4,2021-12-31,2021,12,31,23
205499504,yellow,2021-12-31 23:53:00,2022-01-01 00:02:00,75,75,1.36,540.0,N,True,4,2021-12-31,2021,12,31,23


### 輸出為一份新檔案

In [28]:
custom_output_path = custom_output_path_dfTime

dfTime_TLC.to_parquet(f'{custom_output_path}/dfTime_TLC-{set_data_year}.parquet')
print("done時間:", datetime.datetime.now())

done時間: 2023-08-16 14:40:42.767420


In [29]:
custom_output_path = custom_output_path_dfTLC

df_TLC.to_parquet(f'{custom_output_path}/TLC-{set_data_year}.parquet')
print("done時間:", datetime.datetime.now())

done時間: 2023-08-16 14:43:02.312506
